<a href="https://colab.research.google.com/github/zhangsushen1992/CascadeLearning/blob/master/Cascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install RDKit. Takes 2-3 minutes
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

import rdkit

--2021-01-17 21:17:07--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh [following]
--2021-01-17 21:17:07--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  84.75M   132MB/s    in 0.6s    

2021-01-17 21:17:08 (132 MB/s) - ‘Minicon

In [6]:
%%bash
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
conda config --set always_yes yes --set changeps1 no
conda install -q -y -c conda-forge python=3.7
conda install -q -y -c conda-forge rdkit==2020.09.2 
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem

PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ld_impl_linux-64-2.35.1    |       hea4e1c9_1         617 KB  conda-forge
    python-3.7.9               |hffdb5ce_0_cpython        57.3 MB  conda-forge
    sqlite-3.34.0              |       h74cdb3f_

--2021-01-17 21:22:47--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-01-17 21:22:47--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 41.5M 2s
    50K .......... .......... .......... .......... ..........

In [8]:
import sys
sys.path.append(".")
import os
print(os.getcwd())
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
import argparse
import numpy as np
import tensorflow
import tensorflow.keras as keras
# from tensorflow.keras import layers
# from keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Lambda, Conv1D, GRU
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
# from keras.layers import Dense, Flatten, RepeatVector, Dropout
# from keras.layers import BatchNormalization
# from chemvae import hyperparameters_1
# from chemvae.train_vae import vectorize_data
# from keras import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import *

import copy
import json
from tensorflow.keras.callbacks import CSVLogger, LambdaCallback
import yaml
import rdkit
import mol_utils as mu

/content


In [9]:
def vectorize_data(params):
    # @out : Y_train /Y_test : each is list of datasets.
    #        i.e. if reg_tasks only : Y_train_reg = Y_train[0]
    #             if logit_tasks only : Y_train_logit = Y_train[0]
    #             if both reg and logit_tasks : Y_train_reg = Y_train[0], Y_train_reg = 1
    #             if no prop tasks : Y_train = []

    MAX_LEN = params['MAX_LEN']

    CHARS = yaml.safe_load(open(params['char_file']))
    params['NCHARS'] = len(CHARS)
    NCHARS = len(CHARS)
    CHAR_INDICES = dict((c, i) for i, c in enumerate(CHARS))
    #INDICES_CHAR = dict((i, c) for i, c in enumerate(CHARS))

    ## Load data for properties
    if params['do_prop_pred'] and ('data_file' in params):
        if "data_normalization_out" in params:
            normalize_out = params["data_normalization_out"]
        else:
            normalize_out = None

        ################
        if ("reg_prop_tasks" in params) and ("logit_prop_tasks" in params):
            smiles, Y_reg, Y_logit = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN,
                    reg_tasks=params['reg_prop_tasks'], logit_tasks=params['logit_prop_tasks'],
                    normalize_out = normalize_out)
        elif "logit_prop_tasks" in params:
            smiles, Y_logit = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN,
                    logit_tasks=params['logit_prop_tasks'], normalize_out=normalize_out)
        elif "reg_prop_tasks" in params:
            smiles, Y_reg = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN,
                    reg_tasks=params['reg_prop_tasks'], normalize_out=normalize_out)
        else:
            raise ValueError("please sepcify logit and/or reg tasks")

    ## Load data if no properties
    else:
        smiles = mu.load_smiles_and_data_df(params['data_file'], MAX_LEN)

    if 'limit_data' in params.keys():
        sample_idx = np.random.choice(np.arange(len(smiles)), params['limit_data'], replace=False)
        smiles=list(np.array(smiles)[sample_idx])
        if params['do_prop_pred'] and ('data_file' in params):
            if "reg_prop_tasks" in params:
                Y_reg =  Y_reg[sample_idx]
            if "logit_prop_tasks" in params:
                Y_logit =  Y_logit[sample_idx]

    print('Training set size is', len(smiles))
    print('first smiles: \"', smiles[0], '\"')
    print('total chars:', NCHARS)

    print('Vectorization...')
    X = mu.smiles_to_hot(smiles, MAX_LEN, params[
                             'PADDING'], CHAR_INDICES, NCHARS)

    print('Total Data size', X.shape[0])
    if np.shape(X)[0] % params['batch_size'] != 0:
        X = X[:np.shape(X)[0] // params['batch_size']
              * params['batch_size']]
        if params['do_prop_pred']:
            if "reg_prop_tasks" in params:
                Y_reg = Y_reg[:np.shape(Y_reg)[0] // params['batch_size'] * params['batch_size']]
            if "logit_prop_tasks" in params:
                Y_logit = Y_logit[:np.shape(Y_logit)[0] // params['batch_size'] * params['batch_size']]

    np.random.seed(params['RAND_SEED'])
    rand_idx = np.arange(np.shape(X)[0])
    np.random.shuffle(rand_idx)

    TRAIN_FRAC = 1 - params['val_split']
    num_train = int(X.shape[0] * TRAIN_FRAC)

    if num_train % params['batch_size'] != 0:
        num_train = num_train // params['batch_size'] * \
            params['batch_size']

    train_idx, test_idx = rand_idx[: int(num_train)], rand_idx[int(num_train):]

    if 'test_idx_file' in params.keys():
        np.save(params['test_idx_file'], test_idx)

    X_train, X_test = X[train_idx], X[test_idx]
    print('shape of input vector : {}', np.shape(X_train))
    print('Training set size is {}, after filtering to max length of {}'.format(
        np.shape(X_train), MAX_LEN))

    if params['do_prop_pred']:
        # !# add Y_train and Y_test here
        Y_train = []
        Y_test = []
        if "reg_prop_tasks" in params:
            Y_reg_train, Y_reg_test = Y_reg[train_idx], Y_reg[test_idx]
            Y_train.append(Y_reg_train)
            Y_test.append(Y_reg_test)
        if "logit_prop_tasks" in params:
            Y_logit_train, Y_logit_test = Y_logit[train_idx], Y_logit[test_idx]
            Y_train.append(Y_logit_train)
            Y_test.append(Y_logit_test)

        return X_train, X_test, Y_train, Y_test

    else:
        return X_train, X_test

In [10]:
import json
from collections import OrderedDict


def load_params(param_file=None, verbose=True):
    # Parameters from params.json and exp.json loaded here to override parameters set below
    if param_file is not None:
        hyper_p = json.loads(open(param_file).read(),
                             object_pairs_hook=OrderedDict)
        if verbose:
            print('Using hyper-parameters:')
            for key, value in hyper_p.items():
                print('{:25s} - {:12}'.format(key, str(value)))
            print('rest of parameters are set as default')
    parameters = {

        # for starting model from a checkpoint
        'reload_model': False,
        'prev_epochs': 0,

        # general parameters
        'batch_size': 1,
        'epochs': 1,
        'val_split': 0.1, #validation split
        'loss': 'categorical_crossentropy', # set reconstruction loss

        # convolution parameters
        'batchnorm_conv': True,
        'conv_activation': 'tanh',
        'conv_depth': 4, #4,
        'conv_dim_depth': 8,
        'conv_dim_width': 8,
        'conv_d_growth_factor': 1,#1.15875438383,
        'conv_w_growth_factor': 1,#1.1758149644,

        # decoder parameters
        'gru_depth': 5,#4
        'rnn_activation': 'tanh',
        'recurrent_dim': 35,
        'do_tgru': False,                # use custom terminal gru layer 
        'terminal_GRU_implementation': 0, # use CPU intensive implementation; other implementation modes (1 - GPU, 2- memory) are not yet implemented
        'tgru_dropout': 0.0,
        'temperature': 1.00,            # amount of noise for sampling the final output 

        # middle layer parameters 
        'hg_growth_factor': 1,#1.4928245388, # growth factor applied to determine size of next middle layer.
        'hidden_dim': 50,#100,
        'middle_layer': 2,
        'dropout_rate_mid': 0.0,
        'batchnorm_mid': True,          # apply batch normalization to middle layers
        'activation': 'tanh',

        # Optimization parameters
        'lr': 0.000312087049936,
        'momentum': 0.936948773087,
        'optim': 'adam',                # optimizer to be used

        # vae parameters
        'vae_annealer_start': 22,       # Center for variational weigh annealer 
        'batchnorm_vae': False,         # apply batch normalization to output of the variational layer
        'vae_activation': 'tanh',
        'xent_loss_weight': 1.0,        # loss weight to assign to reconstruction error.
        'kl_loss_weight': 1.0,          # loss weight to assing to KL loss
        "anneal_sigmod_slope": 1.0,     # slope of sigmoid variational weight annealer
        "freeze_logvar_layer": False,   # Choice of freezing the variational layer until close to the anneal starting epoch
        "freeze_offset": 1,             # the number of epochs before vae_annealer_start where the variational layer should be unfrozen

        # property prediction parameters:
        'do_prop_pred': False,          # whether to do property prediction
        'prop_pred_depth': 3,
        'prop_hidden_dim': 36,
        'prop_growth_factor': 0.8,      # ratio between consecutive layer in property prediction
        'prop_pred_activation': 'tanh', 
        'reg_prop_pred_loss': 'mse',    # loss function to use with property prediction error for regression tasks
        'logit_prop_pred_loss': 'binary_crossentropy',  # loss function to use with property prediction for logistic tasks 
        'prop_pred_loss_weight': 0.5,
        'prop_pred_dropout': 0.0,
        'prop_batchnorm': True,

        # print output parameters
        "verbose_print": 1,

    }
    # overwrite parameters
    parameters.update(hyper_p)
    return parameters


In [11]:
params = load_params('exp_2.json')
X_train, X_test, Y_train, Y_test = vectorize_data(params)

Using hyper-parameters:
name                      - zinc        
MAX_LEN                   - 120         
data_file                 - 250k_rndm_zinc_drugs_clean.csv
char_file                 - zinc.json   
encoder_weights_file      - zinc_encoder.h5
decoder_weights_file      - zinc_decoder.h5
test_idx_file             - test_idx.npy
history_file              - history.csv 
checkpoint_path           - ./          
do_prop_pred              - True        
TRAIN_MODEL               - True        
ENC_DEC_TEST              - False       
PADDING                   - right       
RAND_SEED                 - 42          
epochs                    - 70          
vae_annealer_start        - 29          
dropout_rate_mid          - 0.08283292970479479
anneal_sigmod_slope       - 0.5106654305791392
recurrent_dim             - 35          
batch_size                - 126         
lr                        - 0.00039192162392520126
hidden_dim                - 196         
tgru_dropout              -

NumExpr defaulting to 2 threads.


Training set size is 249455
first smiles: " CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1 "
total chars: 35
Vectorization...
Total Data size 249455
shape of input vector : {} (224406, 120, 35)
Training set size is (224406, 120, 35), after filtering to max length of 120


In [12]:
def layer_model(params):
    totallayer=(params['conv_depth']*4+3+params['middle_layer']*3*2+params['gru_depth'])
    layer=[None]*9
    # ----------------- Input layer-----------------
    layer[0] = tensorflow.keras.layers.InputLayer(input_shape=(params['MAX_LEN'], params['NCHARS']), name='input_molecule_smi')
    # layer[0] = Input(shape=(params['MAX_LEN'], params['NCHARS']),name='input_molecule_smi')
    print('layer1')
    # -----------------Encoder layers with batch normalisation-----------------
    layer[1] = tensorflow.keras.layers.Conv1D(int(params['conv_dim_depth']),
                      int(params['conv_dim_width']),
                      activation='tanh',
                      name="encoder_conv0")
 
    layer[2] = tensorflow.keras.layers.BatchNormalization(axis=-1, name="encoder_norm0")
    layer[3] = tensorflow.keras.layers.Conv1D(int(params['conv_dim_depth']),int(params['conv_dim_width']),activation='tanh',
                            name="encoder_conv1")    
    layer[4] = tensorflow.keras.layers.BatchNormalization(axis=-1, name="encoder_norm1")
    layer[5] = tensorflow.keras.layers.Conv1D(int(params['conv_dim_depth']),int(params['conv_dim_width']),activation='tanh',
                            name="encoder_conv2") 
    layer[6] = tensorflow.keras.layers.BatchNormalization(axis=-1, name="encoder_norm2")
    layer[7] = tensorflow.keras.layers.Conv1D(int(params['conv_dim_depth']),int(params['conv_dim_width']),activation='tanh',
                            name="encoder_conv3") 
    layer[8] = tensorflow.keras.layers.BatchNormalization(axis=-1, name="encoder_norm3")
    #-----------------Return the model as a list of different layers objects-----------------
    return layer
 

In [13]:
def decoder_model(params):
    dlayer=[None]*6
    # -----------------Flatten the middle layers to input to GRUS-----------------
    dlayer[0]=Flatten(name='decoder_flatten')
    #-----------------Repeat the vectors to match the length required for GRU-----------------
    dlayer[1] = RepeatVector(120,name='decoder_repeat')

    #-----------------Decoder defined through GRU layers -----------------
   
    dlayer[2] = tensorflow.keras.layers.GRU(params['recurrent_dim'],
                    return_sequences=True, activation='tanh',
                    name="decoder_gru0")
    dlayer[3] = tensorflow.keras.layers.GRU(params['recurrent_dim'],
                    return_sequences=True, activation='tanh',
                    name="decoder_gru1")
    dlayer[4] = tensorflow.keras.layers.GRU(params['recurrent_dim'],
                    return_sequences=True, activation='tanh',
                    name="decoder_gru2")
    dlayer[5]= tensorflow.keras.layers.GRU(params['NCHARS'],
                        return_sequences=True, activation='softmax',
                        name='decoder_gru_final')
    return dlayer

In [14]:
elayer = layer_model(params)
dlayer = decoder_model(params)

layer1


In [18]:
def Cascade(params, encoder_layer, X_train, Y_train, X_test, Y_test,epochs=20,loss='mse',
                        windowSize=5,batch_size=128,outNeurons=64,nb_classes=10,index=0,fast=True,gradient=False):
    '''
    This function performs cascade learning to the autoencoders defined in the argument training_layer
    '''
    #-----------------Define key variables-----------------
    nextModelToTrain = list() 
    saveEncoderLayersIndexes = [1,3,5,7]
    saveDecoderLayersIndexes = [9,10,11,12,13]
    history = dict()
    ModelToTrain=tensorflow.keras.models.Sequential()
    # decoderModel=list()
   
    #-----------------The model starts to append blocks of layers-----------------
    for i in range(0,4): 
        # =========================================
        decoderModel=list()
        # =========================================
        print('The training round is',i)
        if ('iter' + str(i) not in history.keys()): 
            history['iter' + str(i)] = dict() 

        currentAddLayer = list()
        if (i == 0):
            for j in encoder_layer[0:3]: 
                # nextModelToTrain.append(j)
                currentAddLayer.append(j)
            # for j in training_layer[3:5]: 
            #     nextModelToTrain.append(j)
       
           
        else:
            k= encoder_layer[i*2+1:i*2+3] #GET THE LAYERS OF NEXT MODEL TO TRAIN
            # nextModelToTrain.append(3,k)
            currentAddLayer.append(k)

        decoder_layer = decoder_model(params)
        for j in decoder_layer[0:6]: 
                decoderModel.append(j)
       

        # nextModelToTrainInputs = Input(shape=(params['MAX_LEN'], params['NCHARS']), name='input_smi')
        
        #-----------------Starts to add layers to the sequential model----------------
        for currentlayer in currentAddLayer:
            if i==0:
                ModelToTrain.add(currentlayer)
                print('Currently adding',currentlayer)
            else:
                for nni in range(5):
                    ModelToTrain.pop()
                    print('Deleting...%d'%nni,ModelToTrain.summary())
                for layer in ModelToTrain.layers:
                    layer.trainable=False
                print('i=%d,before add'%i,ModelToTrain.summary())
                print('Currently adding',currentlayer)
                for nn_layers in currentlayer:
                    ModelToTrain.add(nn_layers)
                print('i=%d,after add'%i,ModelToTrain.summary())
        for deindex,currentlayer in enumerate(decoderModel):
            print('i=%d,deindex=%d,before add'%(i,deindex),ModelToTrain.summary())
            print('Currently adding deindex%d'%deindex,currentlayer)
            ModelToTrain.add(currentlayer)
            print('i=%d,after add'%i,ModelToTrain.summary())
        #-----------------Compiling the model-----------------
        optimizer = Adam(lr=0.01)
        print('start compiling')
        ModelToTrain.compile(loss=loss,optimizer=optimizer,metrics=['mse'])
        print('finished compiling')
        print(ModelToTrain.summary())
        # ------------------------checkpoint----------------------------
        filepath="weights-improvement-iteration-{i:01d}-{epoch:02d}-{mse:.2f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint]
         # ----------------- Fitting of the model begins here-----------------
        print('Fitting begins .....')
        
        tmpHistory=ModelToTrain.fit(X_train, X_train,
                            batch_size=params['batch_size'],
                            epochs=70,#params['epochs'],
                            initial_epoch=params['prev_epochs'],
                            callbacks=callbacks_list,
                            verbose=params['verbose_print'],
                            validation_data=(X_test, X_test))
        print('Fitting ends ...')
        #-----------------Setting the newly added layers to be trainable -----------------
        #-----------------and the already present layers to be not trainable-----------------
        
        # for layer in ModelToTrain.layers:
        #     layer.trainable=False

        #-----------------Printing key results of the model-----------------
#         print('Weigths are', ModelToTrain.get_weights())

        
        history['iter'+str(i)]['lossTraining'] = tmpHistory.history['loss']
        # history['iter'+str(i)]['accuracyTraining'] = tmpHistory.history['accuracy']
        history['iter'+str(i)]['lossVal'] = tmpHistory.history['val_loss']
        # history['iter'+str(i)]['accuracyVal'] = tmpHistory.history['val_accuracy']
        print('iter',str(i), history['iter'+str(i)])
        
    return ModelToTrain, history


In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint
model, history = Cascade(params=params, encoder_layer=elayer, X_train=X_train, Y_train=Y_train, X_test=X_test, Y_test=Y_test)

The training round is 0
Currently adding <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd982d6c780>
Currently adding <tensorflow.python.keras.layers.convolutional.Conv1D object at 0x7fd981521c88>
Currently adding <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fd982cefcf8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_conv0 (Conv1D)       (None, 113, 8)            2248      
_________________________________________________________________
encoder_norm0 (BatchNormaliz (None, 113, 8)            32        
Total params: 2,280
Trainable params: 2,264
Non-trainable params: 16
_________________________________________________________________
i=0,deindex=0,before add None
Currently adding deindex0 <tensorflow.python.keras.layers.core.Flatten object at 0x7fd9786082e8>
Model: "sequential_1"
___________________________________

KeyError: ignored